In [ ]:
exit()

In [ ]:
%cd 'tunnel_fissure'

In [ ]:
!ls

# Unbox AI
- unbox opensource projects and products of Artificial Intelligence


## welcome to subscribe my channel
- [youtube channel](https://youtube.com/channel/UCAebg3DDFtidQJ0Jp20kyaw)
- [bilibili channel](https://space.bilibili.com/326361150)

# Unbox 'detecting tunnel fissure'




## video
- youtube

[![youtube](https://res.cloudinary.com/marcomontalbano/image/upload/v1603988479/video_to_markdown/images/youtube--dhTfCvxoyPU-c05b58ac6eb4c4700831b2b3070cd403.jpg)](https://www.youtube.com/watch?v=dhTfCvxoyPU "youtube")

## 1.mount google drive folder

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2.clone repo to google drive folder

In [ ]:
%cd '/content/drive/My Drive'
!git clone 'https://github.com/dyh/unbox_detecting_tunnel_fissure.git' 'tunnel_fissure'
%cd 'tunnel_fissure'

## 3.install dependencies

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 'pycocotools>=2.0.1'
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

---

***please make sure of that you have click the [ RESTART RUNTIME ] button -> [ YES ] button to restart colab runtime***

---

## 4.annotate your own data (optional)

### here's how to annotate from 0 to 1, if you don't care about annotation, you can ignore this section.

1. i use the 'google drive backup and sync' app, which automatically synchronizes the google drive files on my machine for easy annotation. you can download it at https://www.google.com/drive/download/

2. go to '/content/drive/My Drive/tunnel_fissure/images/train' folder, and backup the origin 'via_region_data.json' file, change its name to 'via_region_data_bak.json'

3. go to VGG Image Annotator (VIA for short) website http://www.robots.ox.ac.uk/~vgg/software/via/via_demo.html

4. remove 2 demo images in 'VIA'

5. add your images to 'VIA', now we add images from train folder

6. config attributes of Region Attributes

      i. remove 'image_quality' attribute

      ii. change default value of 'name' attribute, from 'not_defined' to 'fissure'

      iii. add 'fissure' and 'water' to 'type' attribute, remove other values

      iv. annotate some fissure regions and water regions

      v. click [ Project -> Save ] to save project file 'project.json', 

      vi. click [ Annotation -> Export Annotations (as json) ] to export json file 'data_json.josn', change its name to 'via_region_data.json'

      vii. we could use 'google drive backup and sync' app to sync 'via_region_data.json' file from download folder to '/content/drive/My Drive/tunnel_fissure/images/train' folder

7. now you have your own training dataset


## 5.import modules

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## 6.register train & val dataset

In [ ]:
from detectron2.structures import BoxMode

def get_fissures_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        list_annos = v["regions"]

        objs = []
        # for _, anno in annos.items():
        for dict_anno in list_annos:
            # assert not anno["region_attributes"]
            anno = dict_anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            # get type from region_attributes to set different category_id
            attr1 = dict_anno["region_attributes"]
            type1 = attr1["type"]

            if type1 == "fissure":
                cat_id = 0
            elif type1 == "water":
                cat_id = 1
            else:
                cat_id = 0

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": cat_id,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["train", "val"]:
    DatasetCatalog.register("fissures_" + d, lambda d=d: get_fissures_dicts(os.path.join("/content/drive/My Drive/tunnel_fissure/images", d)))
    MetadataCatalog.get("fissures_" + d).set(thing_classes=["fissure","water"])
    
fissures_metadata = MetadataCatalog.get("fissures_train")


## 7.preview train dataset

In [ ]:
dataset_dicts = get_fissures_dicts("/content/drive/My Drive/tunnel_fissure/images/train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=fissures_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

## 8.train

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("fissures_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025   # pick a good LR
cfg.SOLVER.MAX_ITER = 300   # you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # default: 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2   # has two classes(fissure, water).

cfg.OUTPUT_DIR = '/content/drive/My Drive/tunnel_fissure/output'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
# trainer.resume_or_load(resume=True)
trainer.train()

print('train done.')

# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/My Drive/tunnel_fissure/output'

## 9.predict images

In [ ]:
from detectron2.utils.visualizer import ColorMode

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.OUTPUT_DIR = '/content/drive/My Drive/tunnel_fissure/output'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

test_image_folder = '/content/drive/My Drive/tunnel_fissure/test'
files = os.listdir(test_image_folder)
# sort by file name
files.sort()

for file_name in files:
    # filter jpg files
    if file_name[-4:] == '.jpg':
        image_path = os.path.join(test_image_folder, file_name)

        # load the origin image
        im = cv2.imread(image_path)
        
        outputs = predictor(im)   # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
        v = Visualizer(im[:, :, ::-1],
                      metadata=fissures_metadata, 
                      scale=0.5,   # zoom out image
                      instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        image_obj = out.get_image()[:, :, ::-1]
        cv2.imwrite(os.path.join(cfg.OUTPUT_DIR, file_name), image_obj)
        cv2_imshow(image_obj)

print('predict done.')


done.
